In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade tensorflow keras protobuf


  Using cached protobuf-6.30.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)


In [8]:
pip install tf-keras

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import re
import string
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

In [10]:
# Load IMDb dataset
dataset, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_data, test_data = dataset["train"], dataset["test"]

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [12]:
# Preprocessing function
def preprocess_text(text):
    text = text.numpy().decode("utf-8").lower()  # Convert to lowercase
    text = re.sub(f"[{string.punctuation}]", "", text)  # Remove punctuation
    return text

# Prepare training and testing data
train_texts, train_labels = [], []
test_texts, test_labels = [], []


In [14]:
for text, label in train_data:
    train_texts.append(preprocess_text(text))
    train_labels.append(label.numpy())

for text, label in test_data:
    test_texts.append(preprocess_text(text))
    test_labels.append(label.numpy())

In [16]:
# Tokenization and Padding
max_length = 200
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length, return_tensors='tf')

train_input_ids = train_encodings["input_ids"]
test_input_ids = test_encodings["input_ids"]

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [18]:
# Load pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [22]:
optimizers = Adam(learning_rate=2e-5, epsilon=1e-08)
losses = SparseCategoricalCrossentropy(from_logits=True)
metric = SparseCategoricalAccuracy("accuracy")
model.compile()

# Train model
history = model.fit(train_input_ids, train_labels, validation_data=(test_input_ids, test_labels), epochs=3, batch_size=16)

print("BERT Model Training Complete!")

Epoch 1/3



1563/1563 [==============================] - 16419s 10s/step - loss: 0.7054 - val_loss: 0.6959
Epoch 2/3
1563/1563 [==============================] - 16566s 11s/step - loss: 0.6974 - val_loss: 0.6932
Epoch 3/3
1563/1563 [==============================] - 16999s 11s/step - loss: 0.6964 - val_loss: 0.6932
BERT Model Training Complete!


In [1]:
y_pred_logits = model.predict(test_input_ids).logits
y_pred = np.argmax(y_pred_logits, axis=1)

print("Classification Report:")
print(classification_report(test_labels, y_pred, target_names=["Negative", "Positive"]))

print("BERT Model Training and Evaluation Complete!")


NameError: name 'model' is not defined